In [1]:
%logstop
%logstart -rtq ~/.logs/pw.py append
%matplotlib inline
import matplotlib
import numpy as np
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [5]:
from static_grader import grader

# PW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to use basic Python data structures, define functions, and control program flow. We will be using these concepts to perform some fundamental data wrangling tasks such as joining data sets together, splitting data into groups, and aggregating data into summary statistics.
**Please do not use `pandas` or `numpy` to answer these questions.**

We will be working with medical data from the British NHS on prescription drugs. Since this is real data, it contains many ambiguities that we will need to confront in our analysis. This is commonplace in data science, and is one of the lessons you will learn in this miniproject.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [6]:
%%bash
mkdir pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/201701scripts_sample.json.gz -nc -P ./pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/practices.json.gz -nc -P ./pw-data

mkdir: cannot create directory ‘pw-data’: File exists
File ‘./pw-data/201701scripts_sample.json.gz’ already there; not retrieving.

File ‘./pw-data/practices.json.gz’ already there; not retrieving.



## Loading the data

The first step of the project is to read in the data. We will discuss reading and writing various kinds of files later in the course, but the code below should get you started.

In [7]:
import gzip
import json
#import simplejson as json

In [8]:
with gzip.open('./pw-data/201701scripts_sample.json.gz', 'rb') as f:
    scripts = json.load(f)

with gzip.open('./pw-data/practices.json.gz', 'rb') as f:
    practices = json.load(f)

This data set comes from Britain's National Health Service. The `scripts` variable is a list of prescriptions issued by NHS doctors. Each prescription is represented by a dictionary with various data fields: `'practice'`, `'bnf_code'`, `'bnf_name'`, `'quantity'`, `'items'`, `'nic'`, and `'act_cost'`. 

In [9]:
scripts[:2]

[{'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000},
 {'bnf_code': '0101021B0AAAHAH',
  'items': 1,
  'practice': 'N81013',
  'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F',
  'nic': 1.95,
  'act_cost': 1.82,
  'quantity': 500}]

A [glossary of terms](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10686/Download-glossary-of-terms-for-GP-prescribing---presentation-level/pdf/PLP_Presentation_Level_Glossary_April_2015.pdf/) and [FAQ](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10048/FAQs-Practice-Level-Prescribingpdf/pdf/PLP_FAQs_April_2015.pdf/) is available from the NHS regarding the data. Below we supply a data dictionary briefly describing what these fields mean.

| Data field |Description|
|:----------:|-----------|
|`'practice'`|Code designating the medical practice issuing the prescription|
|`'bnf_code'`|British National Formulary drug code|
|`'bnf_name'`|British National Formulary drug name|
|`'quantity'`|Number of capsules/quantity of liquid/grams of powder prescribed|
| `'items'`  |Number of refills (e.g. if `'quantity'` is 30 capsules, 3 `'items'` means 3 bottles of 30 capsules)|
|  `'nic'`   |Net ingredient cost|
|`'act_cost'`|Total cost including containers, fees, and discounts|

The `practices` variable is a list of member medical practices of the NHS. Each practice is represented by a dictionary containing identifying information for the medical practice. Most of the data fields are self-explanatory. Notice the values in the `'code'` field of `practices` match the values in the `'practice'` field of `scripts`.

In [10]:
practices[:2]

[{'code': 'A81001',
  'name': 'THE DENSHAM SURGERY',
  'addr_1': 'THE HEALTH CENTRE',
  'addr_2': 'LAWSON STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 1HU'},
 {'code': 'A81002',
  'name': 'QUEENS PARK MEDICAL CENTRE',
  'addr_1': 'QUEENS PARK MEDICAL CTR',
  'addr_2': 'FARRER STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 2AW'}]

In the following questions we will ask you to explore this data set. You may need to combine pieces of the data set together in order to answer some questions. Not every element of the data set will be used in answering the questions.

## Question 1: summary_statistics

Our beneficiary data (`scripts`) contains quantitative data on the number of items dispensed (`'items'`), the total quantity of item dispensed (`'quantity'`), the net cost of the ingredients (`'nic'`), and the actual cost to the patient (`'act_cost'`). Whenever working with a new data set, it can be useful to calculate summary statistics to develop a feeling for the volume and character of the data. This makes it easier to spot trends and significant features during further stages of analysis.

Calculate the sum, mean, standard deviation, and quartile statistics for each of these quantities. Format your results for each quantity as a list: `[sum, mean, standard deviation, 1st quartile, median, 3rd quartile]`. We'll create a `tuple` with these lists for each quantity as a final result.

In [11]:

import math as mt
def describe(key):

    total = 0
    avg = 0
    s = 0
    q25 = 0
    med = 0
    q75 = 0
    L1 = []
    N = len(scripts)
    for i in range(N):
        L = scripts[i]
        L1.append(L[key])
    total=sum(L1)
    avg = total/float(N)
    for i in range(N):
        s+=(L1[i]-avg)**2
    s = mt.sqrt(s/float(N))
    L1.sort()
    N1 = round(N/4)-1
    N2 = round(N/2)-1
    N3 = round((3*N)/4)-1
    q25 = L1[N1]
    med = L1[N2]
    q75 = L1[N3]
    
    
    return (total, avg, s, q25, med, q75)

In [12]:
summary = [('items', describe('items')),
           ('quantity', describe('quantity')),
           ('nic', describe('nic')),
           ('act_cost', describe('act_cost'))]


In [56]:
summary

[('items', (4410054, 11.522744731217633, 33.11216633980368, 1, 3, 8)),
 ('quantity',
  (316356836, 826.5883059943667, 3872.1810146096263, 30, 120, 466)),
 ('nic',
  (29048309.790000338,
   75.89844899484315,
   197.5728266277507,
   7.7,
   22.62,
   65.94)),
 ('act_cost',
  (27053937.599999707,
   70.68748295124895,
   183.26731895303854,
   7.25,
   21.24,
   61.53))]

In [13]:
grader.score.pw__summary_statistics(summary)

Your score:  1.0


## Question 2: most_common_item

Often we are not interested only in how the data is distributed in our entire data set, but within particular groups -- for example, how many items of each drug (i.e. `'bnf_name'`) were prescribed? Calculate the total items prescribed for each `'bnf_name'`. What is the most commonly prescribed `'bnf_name'` in our data?

To calculate this, we first need to split our data set into groups corresponding with the different values of `'bnf_name'`. Then we can sum the number of items dispensed within in each group. Finally we can find the largest sum.

We'll use `'bnf_name'` to construct our groups. You should have *5619* unique values for `'bnf_name'`.

In [14]:
#bnf_names = ...
#assert(len(bnf_names) == 5619)

In [15]:
D = set()
N = len(scripts)
for i in range(N):
    L = scripts[i]
    D.add(L['bnf_name'])
bnf_names = list(D)
assert(len(bnf_names) == 5619)

We want to construct "groups" identified by `'bnf_name'`, where each group is a collection of prescriptions (i.e. dictionaries from `scripts`). We'll construct a dictionary called `groups`, using `bnf_names` as the keys. We'll represent a group with a `list`, since we can easily append new members to the group. To split our `scripts` into groups by `'bnf_name'`, we should iterate over `scripts`, appending prescription dictionaries to each group as we encounter them.

In [16]:
#groups = {name: [] for name in bnf_names}
#for script in scripts:
    # INSERT ...

In [17]:
groups = {name: [] for name in bnf_names}
for script in scripts:
    key = script['bnf_name']
    ite = script['items']
    groups[key].append(ite)
#print(groups)

Now that we've constructed our groups we should sum up `'items'` in each group and find the `'bnf_name'` with the largest sum. The result, `max_item`, should have the form `[(bnf_name, item total)]`, e.g. `[('Foobar', 2000)]`.

In [18]:
#max_item = [("", 0)]

In [19]:
max_item = []
m=0
list_tuple =[]
for key in groups:
    t= tuple()
    t = key, sum(groups[key])
    list_tuple.append(t)
for i in range(len(list_tuple)):
    x,y = list_tuple[i]
    if y>=m:
        r,m = x,y
tu=(r,m)
max_item.append(tu)
print(max_item)
print(len(list_tuple))

[('Omeprazole_Cap E/C 20mg', 113826)]
5619


**TIP:** If you are getting an error from the grader below, please make sure your answer conforms to the correct format of `[(bnf_name, item total)]`.

In [20]:
grader.score.pw__most_common_item(max_item)

Your score:  1.0


**Challenge:** Write a function that constructs groups as we did above. The function should accept a list of dictionaries (e.g. `scripts` or `practices`) and a tuple of fields to `groupby` (e.g. `('bnf_name')` or `('bnf_name', 'post_code')`) and returns a dictionary of groups. The following questions will require you to aggregate data in groups, so this could be a useful function for the rest of the miniproject.

In [21]:
#def group_by_field(data, fields):
   # groups = {}
   # return groups

In [22]:
#groups = group_by_field(scripts, ('bnf_name',))
#test_max_item = ...

#assert test_max_item == max_item

## Question 3: postal_totals

Our data set is broken up among different files. This is typical for tabular data to reduce redundancy. Each table typically contains data about a particular type of event, processes, or physical object. Data on prescriptions and medical practices are in separate files in our case. If we want to find the total items prescribed in each postal code, we will have to _join_ our prescription data (`scripts`) to our clinic data (`practices`).

Find the total items prescribed in each postal code, representing the results as a list of tuples `(post code, total items prescribed)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes. Only include post codes if there is at least one prescription from a practice in that post code.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code.

We can join `scripts` and `practices` based on the fact that `'practice'` in `scripts` matches `'code'` in `practices`. However, we must first deal with the repeated values of `'code'` in `practices`. We want the alphabetically first postal codes.

In [23]:
#practice_postal = {}
#for practice in practices:
  #  if practice['code'] in practice_postal:
  #      practice_postal[practice['code']] = ...
  #  else:
  #      practice_postal[practice['code']] = ...

**Challenge:** This is an aggregation of the practice data grouped by practice codes. Write an alternative implementation of the above cell using the `group_by_field` function you defined previously.

In [24]:
#assert practice_postal['K82019'] == 'HP21 8TR'

Now we can join `practice_postal` to `scripts`.

In [25]:
#joined = scripts[:]
#for script in joined:
#    script['post_code'] = ...

Finally we'll group the prescription dictionaries in `joined` by `'post_code'` and sum up the items prescribed in each group, as we did in the previous question.

In [26]:
#items_by_post = ...

In [27]:
D=set()
for practice in practices:
    D.add(practice['code'])
list_practice=list(D)
list_practice.sort()
groups_code = {name: [] for name in list_practice}
for practice in practices:
    key = practice['code']
    pcode = practice['post_code']
    groups_code[key].append(pcode)
#print(groups_code)

for key in groups_code:
    if len(groups_code[key])>1:
        x = min(groups_code[key])
        groups_code[key]= list()
        groups_code[key].append(x)
for key in groups_code:
    x=groups_code[key][0]
    groups_code[key]=x
        

In [28]:
S = set()
for key in groups_code:
    S.add(groups_code[key])
list_post_code=list(S)
list_post_code.sort()
#print(list_post_code)

practice_postal = dict()
for elment in list_post_code:
    for key in groups_code:
        if groups_code[key]==elment:
            practice_postal[key]=elment

#print(practice_postal)
sorted(practice_postal.values())
#practice_postal['K82019']


['AL1 3HD',
 'AL1 3JB',
 'AL1 4JE',
 'AL10 0BS',
 'AL10 0LF',
 'AL10 0NL',
 'AL10 0UR',
 'AL10 8HP',
 'AL2 1ES',
 'AL2 1ES',
 'AL2 1ES',
 'AL2 3JX',
 'AL3 5ER',
 'AL3 5HB',
 'AL3 5JB',
 'AL3 5JB',
 'AL3 5NF',
 'AL3 5NP',
 'AL3 7BL',
 'AL3 8LJ',
 'AL5 2BT',
 'AL5 4HX',
 'AL5 4QA',
 'AL5 4QA',
 'AL6 9EF',
 'AL6 9SB',
 'AL7 1BW',
 'AL7 1BW',
 'AL7 1JR',
 'AL7 3UJ',
 'AL7 4HL',
 'AL7 4HL',
 'AL7 4HL',
 'AL7 4HL',
 'AL7 4PL',
 'AL8 6JL',
 'AL8 7QG',
 'AL9 7SN',
 'B1 1EQ',
 'B1 3AL',
 'B1 3RA',
 'B10 0BS',
 'B10 0JL',
 'B10 0TU',
 'B10 0UG',
 'B10 9AB',
 'B10 9AB',
 'B10 9QE',
 'B11 1LU',
 'B11 1LU',
 'B11 1TX',
 'B11 3ND',
 'B11 4AN',
 'B11 4AN',
 'B11 4BW',
 'B11 4BW',
 'B11 4BW',
 'B11 4DG',
 'B11 4RA',
 'B12 0UF',
 'B12 0UF',
 'B12 0YA',
 'B12 8HE',
 'B12 8QE',
 'B12 9LP',
 'B12 9LP',
 'B12 9LP',
 'B12 9RR',
 'B13 0HN',
 'B13 0HN',
 'B13 8JL',
 'B13 8JL',
 'B13 8JL',
 'B13 8JL',
 'B13 8JL',
 'B13 8JS',
 'B13 8QS',
 'B13 9HD',
 'B13 9HD',
 'B13 9LH',
 'B14 4DU',
 'B14 4DU',
 'B14 4DU',
 '

In [29]:
len(list_practice)

10843

In [30]:
assert groups_code['K82019'] == 'HP21 8TR'

assert practice_postal['K82019'] == 'HP21 8TR'

post_set = set()
a=0
practice_postal_100 = dict()

for key in practice_postal:
    if a<100:
        practice_postal_100[key] = practice_postal[key]
        post_set.add(practice_postal[key])
    else:
        break
    a = len(post_set)

print(practice_postal)
ad=list(practice_postal)
ad.sort()
print(ad)
print(len(ad))

practice_postal['M85078']

L1=list(practice_postal)
print(len(L1))

set_practice = set()
for script in scripts:
    set_practice.add(script['practice'])
list_practice_scripts = list(set_practice)
print(len(scripts), len(list_practice_scripts))

groups_practice_scripts = {name: [] for name in list_practice_scripts}
for script in scripts:
    key = script['practice']
    ite = script['items']
    groups_practice_scripts[key].append(ite)
#print(groups_practice_scripts)

tuple_practice_scripts =[]
for key in groups_practice_scripts:
    t= tuple()
    t = key, sum(groups_practice_scripts[key])
    tuple_practice_scripts.append(t)
#print((tuple_practice_scripts))
tuple_practice_scripts.sort()
#print('###############################################################')
#print(tuple_practice_scripts)

final = list()
for key in practice_postal:
    for x in tuple_practice_scripts:
        code,ite = x
        if code==key:
            t=tuple()
            t=practice_postal[key],ite
            final.append(t)
print(code,ite)

set_final = set()
for el in final:
    x,y=el
    set_final.add(x)
print(len(set_final))
Lfinal = list(set_final)
Lfinal.sort()
#print(Lfinal)

list_set_final = dict()
for el in Lfinal:
    list_set_final[el]=0
#print(list_set_final)
for key in list_set_final:
    for el in final:
        x,y=el
        if x==key:
            list_set_final[key]=list_set_final[key]+y
#print('################################')
#print(list_set_final)


a=0
postal_total=list()
for key in list_set_final:
    t=tuple()
    t=key,list_set_final[key]
    postal_total.append(t)
postal_totals=postal_total[:100]

print(postal_totals)

{'E82060': 'AL1 3HD', 'E82031': 'AL1 3JB', 'E82004': 'AL1 4JE', 'E82002': 'AL10 0BS', 'Y05146': 'AL10 0LF', 'E82018': 'AL10 0NL', 'Y04223': 'AL10 0UR', 'E82023': 'AL10 8HP', 'E82113': 'AL2 1ES', 'Y01966': 'AL2 1ES', 'Y03231': 'AL2 1ES', 'E82055': 'AL2 3JX', 'E82107': 'AL3 5ER', 'E82084': 'AL3 5HB', 'Y01802': 'AL3 5JB', 'Y04967': 'AL3 5JB', 'E82059': 'AL3 5NF', 'E82014': 'AL3 5NP', 'E82095': 'AL3 7BL', 'E82663': 'AL3 8LJ', 'E82037': 'AL5 2BT', 'E82077': 'AL5 4HX', 'E82071': 'AL5 4QA', 'Y02175': 'AL5 4QA', 'E82019': 'AL6 9EF', 'Y02029': 'AL6 9SB', 'Y03837': 'AL7 1BW', 'Y03838': 'AL7 1BW', 'Y04607': 'AL7 1JR', 'E82040': 'AL7 3UJ', 'Y00415': 'AL7 4HL', 'Y01061': 'AL7 4HL', 'Y02639': 'AL7 4HL', 'Y03580': 'AL7 4HL', 'E82062': 'AL7 4PL', 'Y02176': 'AL8 6JL', 'E82041': 'AL8 7QG', 'E82058': 'AL9 7SN', 'Y04878': 'B1 1EQ', 'M85797': 'B1 3AL', 'M85768': 'B1 3RA', 'M85024': 'B10 0BS', 'M85005': 'B10 0JL', 'M85749': 'B10 0TU', 'M85699': 'B10 0UG', 'M85679': 'B10 9AB', 'Y02544': 'B10 9AB', 'M85108': 

Y05670 15
118
[('B11 4BW', 20673), ('B18 7AL', 19001), ('B21 9RY', 29103), ('B23 6DJ', 24859), ('B70 7AW', 36531), ('BB11 2DL', 34356), ('BB2 1AX', 28254), ('BB3 1PY', 54514), ('BB4 5SL', 29388), ('BB7 2JG', 44585), ('BB8 0JZ', 54380), ('BB9 7SR', 38224), ('BD3 8QH', 21010), ('BH18 8EE', 39413), ('BH23 3AF', 32545), ('BL1 8TU', 26132), ('BL3 5HP', 27147), ('BL9 0NJ', 32062), ('BL9 0SN', 35275), ('CB9 8HF', 51337), ('CH1 4DS', 34915), ('CH65 6TG', 25090), ('CT11 8AD', 44358), ('CV1 4FS', 37210), ('CW1 3AW', 64104), ('CW5 5NX', 38797), ('CW7 1AT', 43164), ('DA1 2HA', 26075), ('DA11 8BZ', 24090), ('DN22 7XF', 43091), ('DN34 4GB', 48013), ('FY2 0JG', 69118), ('FY4 1TJ', 62886), ('FY5 2TZ', 44258), ('FY7 8GU', 34473), ('GL1 3PX', 38120), ('GL50 4DP', 74822), ('GU9 9QS', 32131), ('HA0 4UZ', 22755), ('HA3 7LT', 32113), ('HG1 5AR', 32684), ('HU7 4DW', 49107), ('KT14 6DH', 26758), ('KT6 6EZ', 38975), ('L31 0DJ', 32065), ('L36 7XY', 22965), ('L5 0QW', 24676), ('L7 6HD', 42569), ('LA1 1PN', 47335

In [31]:
#postal_totals = [('B11 4BW', 20673)] * 100

grader.score.pw__postal_totals(postal_totals)

Your score:  1.0


## Question 4: items_by_region

Now we'll combine the techniques we've developed to answer a more complex question. Find the most commonly dispensed item in each postal code, representing the results as a list of tuples (`post_code`, `bnf_name`, amount dispensed as proportion of total). Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** We'll continue to use the `joined` variable we created before, where we've chosen the alphabetically first postal code for each practice. Additionally, some postal codes will have multiple `'bnf_name'` with the same number of items prescribed for the maximum. In this case, we'll take the alphabetically first `'bnf_name'`.

Now we need to calculate the total items of each `'bnf_name'` prescribed in each `'post_code'`. Use the techniques we developed in the previous questions to calculate these totals. You should have 141196 `('post_code', 'bnf_name')` groups.

In [32]:
#total_items_by_bnf_post = ...
#assert len(total_items_by_bnf_post) == 141196

Let's use `total_items` to find the maximum item total for each postal code. To do this, we will want to regroup `total_items_by_bnf_post` by `'post_code'` only, not by `('post_code', 'bnf_name')`. First let's turn `total_items` into a list of dictionaries (similar to `scripts` or `practices`) and then group it by `'post_code'`. You should have 118 groups in the resulting `total_items_by_post` after grouping `total_items` by `'post_code'`.

In [33]:
#total_items = ...
#assert len(total_items_by_post) == 118

Now we will aggregate the groups in `total_by_item_post` to create `max_item_by_post`. Some `'bnf_name'` have the same item total within a given postal code. Therefore, if more than one `'bnf_name'` has the maximum item total in a given postal code, we'll take the alphabetically first `'bnf_name'`. We can do this by [sorting](https://docs.python.org/2.7/howto/sorting.html) each group according to the item total and `'bnf_name'`.

In [34]:
#max_item_by_post = ...

In order to express the item totals as a proportion of the total amount of items prescribed across all `'bnf_name'` in a postal code, we'll need to use the total items prescribed that we previously calculated as `items_by_post`. Calculate the proportions for the most common `'bnf_names'` for each postal code. Format your answer as a list of tuples: `[(post_code, bnf_name, total)]`

In [35]:
#items_by_region = [('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.0341508247)] * 100

In [36]:
print(len(bnf_names)*len(list_practice_scripts))
groups_practice_codepostal = dict()
for key in list_practice_scripts:
    for code in practice_postal:
        if key==code:
            groups_practice_codepostal[code]=practice_postal[code]

2775786


In [37]:

list_100 = list()
for el in postal_totals:
    x,y=el
    list_100.append(x)
#print(list_100)

#len(list_practice_codepostal)
#Combining bnf_names and list_practice_codepostal


In [38]:
list_bnf_post_code = list()

for key in groups_practice_codepostal:
    for name in bnf_names:
        t=tuple()
        t=key,groups_practice_codepostal[key],name
        list_bnf_post_code.append(t)
print(len(list_bnf_post_code))

2775786


In [39]:
groups_practice_scripts_bnf = {name: [] for name in list_practice_scripts}
for script in scripts:
    key = script['practice']
    t=tuple()
    t=script['bnf_name'],script['items']
    groups_practice_scripts_bnf[key].append(t)
print(len(groups_practice_scripts_bnf))

494


In [40]:

s=0
for key in groups_practice_scripts_bnf:
    x=len(groups_practice_scripts_bnf[key])
    s+=x
print(s)

382726


In [41]:
list_tuple_codepost_bnf_item = list()
for key in groups_practice_scripts_bnf:
    t=tuple
    t=practice_postal[key],groups_practice_scripts_bnf[key]
    list_tuple_codepost_bnf_item.append(t)

print(len(list_tuple_codepost_bnf_item))

494


In [42]:
#groups_practice_scripts_bnf['N81013']


In [43]:
#print(list_tuple_codepost_bnf_item)

total_items_by_bnf_post = list()
for el in list_tuple_codepost_bnf_item:
    x,y = el
    new_set = set()
    for el2 in y:
        x2,y2 = el2
        new_set.add(x2)
    l_new = list(new_set)
    l_new.sort()
    for el3 in l_new:
        ite=0
        t=tuple()
        for el4 in y:
            x4,y4=el4
            if x4==el3:
                ite+=y4
        t=x,el3,ite
        total_items_by_bnf_post.append(t)

In [44]:
print((total_items_by_bnf_post[:2]))

[('TR1 2JA', '3m Health Care_Cavilon Durable Barrier C', 10), ('TR1 2JA', '3m Health Care_Cavilon No Sting Barrier', 4)]


In [45]:
l1 = list()
for el in total_items_by_bnf_post:
    x,y,z=el
    t=tuple()
    t=x,y
    l1.append(t)
print(l1[:2])           

[('TR1 2JA', '3m Health Care_Cavilon Durable Barrier C'), ('TR1 2JA', '3m Health Care_Cavilon No Sting Barrier')]


In [46]:
len(l1)

381031

In [47]:
G = zip(range(100),list_100)
G = dict(G)
print(G)

{0: 'B11 4BW', 1: 'B18 7AL', 2: 'B21 9RY', 3: 'B23 6DJ', 4: 'B70 7AW', 5: 'BB11 2DL', 6: 'BB2 1AX', 7: 'BB3 1PY', 8: 'BB4 5SL', 9: 'BB7 2JG', 10: 'BB8 0JZ', 11: 'BB9 7SR', 12: 'BD3 8QH', 13: 'BH18 8EE', 14: 'BH23 3AF', 15: 'BL1 8TU', 16: 'BL3 5HP', 17: 'BL9 0NJ', 18: 'BL9 0SN', 19: 'CB9 8HF', 20: 'CH1 4DS', 21: 'CH65 6TG', 22: 'CT11 8AD', 23: 'CV1 4FS', 24: 'CW1 3AW', 25: 'CW5 5NX', 26: 'CW7 1AT', 27: 'DA1 2HA', 28: 'DA11 8BZ', 29: 'DN22 7XF', 30: 'DN34 4GB', 31: 'FY2 0JG', 32: 'FY4 1TJ', 33: 'FY5 2TZ', 34: 'FY7 8GU', 35: 'GL1 3PX', 36: 'GL50 4DP', 37: 'GU9 9QS', 38: 'HA0 4UZ', 39: 'HA3 7LT', 40: 'HG1 5AR', 41: 'HU7 4DW', 42: 'KT14 6DH', 43: 'KT6 6EZ', 44: 'L31 0DJ', 45: 'L36 7XY', 46: 'L5 0QW', 47: 'L7 6HD', 48: 'LA1 1PN', 49: 'LE10 1DS', 50: 'LE18 2EW', 51: 'LE5 3GH', 52: 'LN2 2JP', 53: 'LS9 9EF', 54: 'M11 4EJ', 55: 'M26 2SP', 56: 'M30 0NU', 57: 'M35 0AD', 58: 'ME8 8AA', 59: 'N9 7HD', 60: 'NE10 9QG', 61: 'NE24 1DX', 62: 'NE37 2PU', 63: 'NE38 7NQ', 64: 'NG7 3GW', 65: 'NG7 5HY', 66: 'N

In [48]:
list_new = list()
for el in total_items_by_bnf_post:
    x,y,z=el
    if x in G.values():
        list_new.append(el)

print(len(list_new))


322072


In [49]:
G2 = { value : [] for value in G.values() }
G2

{'B11 4BW': [],
 'B18 7AL': [],
 'B21 9RY': [],
 'B23 6DJ': [],
 'B70 7AW': [],
 'BB11 2DL': [],
 'BB2 1AX': [],
 'BB3 1PY': [],
 'BB4 5SL': [],
 'BB7 2JG': [],
 'BB8 0JZ': [],
 'BB9 7SR': [],
 'BD3 8QH': [],
 'BH18 8EE': [],
 'BH23 3AF': [],
 'BL1 8TU': [],
 'BL3 5HP': [],
 'BL9 0NJ': [],
 'BL9 0SN': [],
 'CB9 8HF': [],
 'CH1 4DS': [],
 'CH65 6TG': [],
 'CT11 8AD': [],
 'CV1 4FS': [],
 'CW1 3AW': [],
 'CW5 5NX': [],
 'CW7 1AT': [],
 'DA1 2HA': [],
 'DA11 8BZ': [],
 'DN22 7XF': [],
 'DN34 4GB': [],
 'FY2 0JG': [],
 'FY4 1TJ': [],
 'FY5 2TZ': [],
 'FY7 8GU': [],
 'GL1 3PX': [],
 'GL50 4DP': [],
 'GU9 9QS': [],
 'HA0 4UZ': [],
 'HA3 7LT': [],
 'HG1 5AR': [],
 'HU7 4DW': [],
 'KT14 6DH': [],
 'KT6 6EZ': [],
 'L31 0DJ': [],
 'L36 7XY': [],
 'L5 0QW': [],
 'L7 6HD': [],
 'LA1 1PN': [],
 'LE10 1DS': [],
 'LE18 2EW': [],
 'LE5 3GH': [],
 'LN2 2JP': [],
 'LS9 9EF': [],
 'M11 4EJ': [],
 'M26 2SP': [],
 'M30 0NU': [],
 'M35 0AD': [],
 'ME8 8AA': [],
 'N9 7HD': [],
 'NE10 9QG': [],
 'NE24 1DX': [

In [50]:
for key in G2:
    for el in total_items_by_bnf_post:
        x,y,z=el
        if key==x:
            #t=tuple()
            t=y,z
        G2[key].append(t)

In [ ]:
list_max = []
for key in G2:
    s=set()
    for el in G2[key]:
        y,z=el
        s.add(y)
    l=list(s)
    l.sort()
    l2=list()
    for el in l:
        t=tuple()
        ite=0
        for el2 in G2[key]:
            y,z=el2
            if y==el:
                ite+=z
        t=el,ite
        l2.append(t)
    bnf_max='a'
    m=0
    for el in l2:
        y,z=el
        if z>m:
            m=z
            bnf_max=y
    t2=key,bnf_max,m
    list_max.append(t2)


In [ ]:
max_item_by_post = list_max

In [ ]:
items_by_region = list()
for i in range(100):
    t=tuple()
    x,y,z=max_item_by_post[i]
    y2,z2=postal_totals[i]
    z3=z/float(z2)
    t=x,y,z3
    items_by_region.append(t)

In [ ]:
grader.score.pw__items_by_region(items_by_region)

*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*